In [14]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
import os


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls '/content/drive'


MyDrive


In [7]:
#Give dataset path
train_path = '/content/drive/MyDrive/archive (6)/Alzheimer_s Dataset/train'
test_path = '/content/drive/MyDrive/archive (6)/Alzheimer_s Dataset/test'

In [25]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_set = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=40,
    class_mode='categorical'  # Adjust based on your dataset
)

test_set = test_datagen.flow_from_directory(
    test_path,
    target_size=(224, 224),
    batch_size=40,
    class_mode='categorical'  # Adjust based on your dataset
)

Found 5121 images belonging to 4 classes.
Found 1279 images belonging to 4 classes.


In [26]:
# Load Pretrained VGG16 Model
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers in the base model
for layer in vgg16_base.layers:
    layer.trainable = False

# Add custom layers on top of the base model
x = Flatten()(vgg16_base.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)  # Output classes = 4, so using softmax activation

# Create the model
model = Model(inputs=vgg16_base.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])  # Use categorical_crossentropy for multi-class classification

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
# Fine-tune the model
history = model.fit_generator(
    train_set,
    steps_per_epoch=len(train_set),
    epochs=10,
    validation_data=test_set,
    validation_steps=len(test_set)
)

# Extract the final accuracy from the history object
final_accuracy = history.history['accuracy'][-1]
final_accuracy_percentage = final_accuracy * 100  # Convert to percentage

print("Final Accuracy:", final_accuracy_percentage, "%")


<ipython-input-8-c789e222f6ab>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/10
129/129 [==============================] - 3865s 29s/step - loss: 1.3554 - accuracy: 0.5850 - val_loss: 1.2705 - val_accuracy: 0.5395
Epoch 2/10
129/129 [==============================] - 3677s 29s/step - loss: 0.5804 - accuracy: 0.7618 - val_loss: 0.7324 - val_accuracy: 0.6935
Epoch 3/10
129/129 [==============================] - 3712s 29s/step - loss: 0.3760 - accuracy: 0.8586 - val_loss: 1.0117 - val_accuracy: 0.6231
Epoch 4/10
129/129 [==============================] - 3684s 29s/step - loss: 0.2803 - accuracy: 0.9022 - val_loss: 0.7977 - val_accuracy: 0.6904
Epoch 5/10
129/129 [==============================] - 3693s 29s/step - loss: 0.2315 - accuracy: 0.9223 - val_loss: 0.8590 - val_accuracy: 0.6787
Epoch 6/10
129/129 [==============================] - 3690s 29s/step - loss: 0.1920 - accuracy: 0.9365 - val_loss: 1.1548 - val_accuracy: 0.6677
Epoch 7/10
129/129 [==============================] - 3661s 28s/step - loss: 0.1237 - accuracy: 0.9690 - val_loss: 0.8229 - val_ac

In [ ]:
model.save('/content/drive/MyDrive/model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [32]:
import os
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the main dataset directory
dataset_dir = '/content/drive/MyDrive/archive (6)/Alzheimer_s Dataset'

# Define the inner directory for the dataset
inner_dir = dataset_dir  # This would be the main directory containing 'train' and 'test'

# Define paths to the test data directory
test_dir = os.path.join(inner_dir, 'test')

# Check if the test directory exists and print its contents for debugging
if not os.path.exists(test_dir):
    print(f"The directory {test_dir} does not exist.")
else:
    print(f"The directory {test_dir} exists.")
    # List directory contents for debugging
    for root, dirs, files in os.walk(test_dir):
        print(root, "contains", len(dirs), "subdirectories and", len(files), "files.")

# Define image size and batch size
img_size = (224, 224)
batch_size = 40

# Create ImageDataGenerators for test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Create the test data generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  # Assuming images are organized in folders by class
    shuffle=False
)

# Print class indices for debugging
print("Class indices:", test_generator.class_indices)

# Load Pretrained VGG16 Model
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers in the base model
for layer in vgg16_base.layers:
    layer.trainable = False

# Add custom layers on top of the base model
x = Flatten()(vgg16_base.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)  # Output classes = 4, so using softmax activation

# Create the model
model = Model(inputs=vgg16_base.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])  # Use categorical_crossentropy for multi-class classification

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_generator))
print("Test Accuracy:", test_accuracy)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
The directory /content/drive/MyDrive/archive (6)/Alzheimer_s Dataset/test exists.
/content/drive/MyDrive/archive (6)/Alzheimer_s Dataset/test contains 4 subdirectories and 0 files.
/content/drive/MyDrive/archive (6)/Alzheimer_s Dataset/test/VeryMildDemented contains 0 subdirectories and 448 files.
/content/drive/MyDrive/archive (6)/Alzheimer_s Dataset/test/MildDemented contains 0 subdirectories and 179 files.
/content/drive/MyDrive/archive (6)/Alzheimer_s Dataset/test/ModerateDemented contains 0 subdirectories and 12 files.
/content/drive/MyDrive/archive (6)/Alzheimer_s Dataset/test/NonDemented contains 0 subdirectories and 640 files.
Found 1279 images belonging to 4 classes.
Class indices: {'MildDemented': 0, 'ModerateDemented': 1, 'NonDemented': 2, 'VeryMildDemented': 3}


32/32 [==============================] - 843s 26s/step - loss: 1.2021 - accuracy: 0.4965
Test Accuracy: 0.49648162722587585


In [33]:
model.save('/content/drive/MyDrive/model11.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
